# Goldstein-price function

**Contributed by**: Benoît Legat

In this example, we consider the minimization of the [Goldstein-price function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [1]:
using SumOfSquares
using DynamicPolynomials

Create *symbolic* variables (not JuMP *decision* variables)

In [2]:
@polyvar x[1:2]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂],)

To use Sum-of-Squares Programming, we first need to pick an SDP solver,
see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [3]:
import Clarabel
using Dualization
model = SOSModel(dual_optimizer(Clarabel.Optimizer))

A JuMP Model
├ solver: Dual model with Clarabel attached
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

Create a JuMP decision variable for the lower bound

In [4]:
@variable(model, γ)

γ

`f(x)` is the Goldstein-Price function

In [5]:
f1 = x[1] + x[2] + 1
f2 = 19 - 14*x[1] + 3*x[1]^2 - 14*x[2] + 6*x[1]*x[2] + 3*x[2]^2
f3 = 2*x[1] - 3*x[2]
f4 = 18 - 32*x[1] + 12*x[1]^2 + 48*x[2] - 36*x[1]*x[2] + 27*x[2]^2
f = (1 + f1^2*f2) * (30 + f3^2*f4)

600 + 720x₂ + 720x₁ + 3060x₂² - 4680x₁x₂ + 1260x₁² + 12288x₂³ - 19296x₁x₂² + 7344x₁²x₂ - 1072x₁³ + 14346x₂⁴ - 23616x₁x₂³ + 7776x₁²x₂² + 5784x₁³x₂ - 2454x₁⁴ + 1944x₂⁵ - 11880x₁x₂⁴ + 5040x₁²x₂³ + 9840x₁³x₂² - 7680x₁⁴x₂ + 1344x₁⁵ - 4428x₂⁶ - 1188x₁x₂⁵ + 8730x₁²x₂⁴ + 1240x₁³x₂³ - 5370x₁⁴x₂² - 168x₁⁵x₂ + 952x₁⁶ - 648x₂⁷ + 1944x₁x₂⁶ + 3672x₁²x₂⁵ - 3480x₁³x₂⁴ - 4080x₁⁴x₂³ + 2592x₁⁵x₂² + 1344x₁⁶x₂ - 768x₁⁷ + 729x₂⁸ + 972x₁x₂⁷ - 1458x₁²x₂⁶ - 1836x₁³x₂⁵ + 1305x₁⁴x₂⁴ + 1224x₁⁵x₂³ - 648x₁⁶x₂² - 288x₁⁷x₂ + 144x₁⁸

Constraints `f(x) - γ` to be a sum of squares

In [6]:
con_ref = @constraint(model, f >= γ)
@objective(model, Max, γ)
optimize!(model)

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 121
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : PSDTriangle = 1,  numel = 120

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       g

The lower bound found is 3

In [7]:
solution_summary(model)

solution_summary(; result = 1, verbose = false)
├ solver_name          : Dual model with Clarabel attached
├ Termination
│ ├ termination_status : ALMOST_OPTIMAL
│ ├ result_count       : 1
│ └ raw_status         : ALMOST_SOLVED
├ Solution (result = 1)
│ ├ primal_status        : NEARLY_FEASIBLE_POINT
│ ├ dual_status          : NEARLY_FEASIBLE_POINT
│ ├ objective_value      : 3.00000e+00
│ └ dual_objective_value : 3.00000e+00
└ Work counters
  ├ solve_time (sec)   : 2.09994e-02
  └ barrier_iterations : 21

The moment matrix is as follows, we can already see the global minimizer
`[0, -1]` from the entries `(2, 1)` and `(3, 1)`.
This heuristic way to obtain solutions to the polynomial optimization problem
is suggested in [Laurent2008; (6.15)](@cite).

In [8]:
ν = moment_matrix(con_ref)

MomentMatrix with row/column basis:
 SubBasis{Monomial}([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3, x[2]^4, x[1]*x[2]^3, x[1]^2*x[2]^2, x[1]^3*x[2], x[1]^4])
And entries in a 15×15 SymMatrix{Float64}:
  0.9999999997859073     -0.9999988219054464     …   2.5166009647361593e-7
 -0.9999988219054464      0.9999976841708637         9.47676328056913e-8
  1.1872461215867367e-6  -1.1600287259108588e-6      4.7451356029855384e-7
  0.9999976618226798     -0.9999965064454199         4.8965215088882085e-5
 -1.1599500181679241e-6   1.1855210516213522e-6      4.618762907520152e-5
  5.8216332622599706e-8   3.0356199948443204e-8  …   9.745535477720552e-5
 -0.999996513745934       0.9999953642434687        -0.0005565093093445481
  1.160209683468974e-6   -1.1225671941552976e-6     -0.00032131825703568713
  2.9745760282044875e-8   1.8377122547470757e-9     -0.0008744279872899929
  1.1134410809885912e-7   8.327992018483806e-8      -0.0007650100609308081
  0.9999953

Many entries of the matrix actually have the same moment.
We can obtain the following list of these moments without duplicates
(ignoring when difference of entries representing the same moments is below `1e-5`)

In [9]:
μ = moment_vector(ν, atol = 1e-5)

MomentVector{Float64, SubBasis{Monomial, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}, Vector{Float64}}([0.9999999997859073, -0.9999988219054464, 1.1872461215867367e-6, 0.9999976618226798, -1.1599500181679241e-6, 5.8216332622599706e-8, -0.999996513745934, 1.160209683468974e-6, 2.9745760282044875e-8, 1.1134410809885912e-7  …  -0.0007650100609308081, 1.412089891519067, -0.28776435809882966, 0.4734587642070385, -0.194381193968364, 0.6122725920054648, 0.014996139527614954, 0.9253620565525771, 0.48570399361626404, 1.6309121889456073], SubBasis{Monomial}([1, x₂, x₁, x₂², x₁x₂, x₁², x₂³, x₁x₂², x₁²x₂, x₁³, x₂⁴, x₁x₂³, x₁²x₂², x₁³x₂, x₁⁴, x₂⁵, x₁x₂⁴, x₁²x₂³, x₁³x₂², x₁⁴x₂, x₁⁵, x₂⁶, x₁x₂⁵, x₁²x₂⁴, x₁³x₂³, x₁⁴x₂², x₁

The truncated moment matrix can then be obtained as follows

In [10]:
ν_truncated = moment_matrix(μ, monomials(x, 0:3))

MomentMatrix with row/column basis:
 SubBasis{Monomial}([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3])
And entries in a 10×10 SymMatrix{Float64}:
  0.9999999997859073     -0.9999988219054464     …  1.1134410809885912e-7
 -0.9999988219054464      0.9999976618226798        8.928243272967077e-8
  1.1872461215867367e-6  -1.1599500181679241e-6     2.5166009647361593e-7
  0.9999976618226798     -0.999996513745934         3.5488281830330126e-8
 -1.1599500181679241e-6   1.160209683468974e-6      9.47676328056913e-8
  5.8216332622599706e-8   2.9745760282044875e-8  …  4.7451356029855384e-7
 -0.999996513745934       0.9999953885900964        1.4340818755853253e-5
  1.160209683468974e-6   -1.0827791561456249e-6     4.8965215088882085e-5
  2.9745760282044875e-8   2.913694878100692e-8      4.618762907520152e-5
  1.1134410809885912e-7   8.928243272967077e-8      9.745535477720552e-5

Let's check if the flatness property is satisfied.
The rank of `ν_truncated` seems to be 1:

In [11]:
using LinearAlgebra
LinearAlgebra.svdvals(Matrix(ν_truncated.Q))
LinearAlgebra.rank(Matrix(ν_truncated.Q), rtol = 1e-3)
svdvals(Matrix(ν_truncated.Q))

10-element Vector{Float64}:
 3.999990725389048
 0.00014941337488010636
 3.8463103544774386e-5
 5.00647368410897e-7
 9.73705614060403e-8
 4.8867451285657545e-8
 4.067233993284245e-8
 3.755056514424653e-8
 3.60552441531946e-8
 3.139767073098475e-9

The rank of `ν` is clearly higher than 1, closer to 3:

In [12]:
svdvals(Matrix(ν.Q))

15-element Vector{Float64}:
 5.141887449326348
 2.4407594264687362
 1.4710241419307037
 0.0004074855572430628
 0.00013845129928262667
 3.543119975228378e-5
 2.1610651138115324e-5
 5.913752227517728e-7
 3.0867009566270523e-8
 2.783871660796074e-8
 4.871621314580938e-9
 2.831448387851015e-9
 6.916635365345059e-10
 3.327529933099593e-10
 1.9424275214671552e-11

Even if the flatness property is not satisfied, we can
still try extracting the minimizer with a low rank decomposition of rank 3.
We find the optimal solution again doing so:

In [13]:
atomic_measure(ν, FixedRank(3))

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [1.1821785890260018e-6, -0.999998831548167] with weight 1.0274960457332836

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*